In [1]:
model_name = "tokyotech-llm/Llama-3.1-Swallow-8B-Instruct-v0.2"
#model_name = "meta-llama/Llama-3.1-8B-Instruct"
#model_name = "cyberagent/calm3-22b-chat"

In [2]:
%pip install --upgrade transformers
%pip install --upgrade accelerate
%pip install torch  bitsandbytes huggingface_hub[cli] huggingface_hub hf_transfer
%pip install python-dotenv

  Using cached transformers-4.47.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.47.1-py3-none-any.whl (10.1 MB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.1

In [4]:
import os
import re
import requests
import json
import pickle
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from huggingface_hub import login
from huggingface_hub import snapshot_download
from transformers.utils import move_cache

# .envに書き込んだ hugging faceを読み込むには、以下のコード
#from dotenv import load_dotenv
#load_dotenv()
#HUGGING_FACE_TOKEN = os.getenv('HUGGING_FACE_TOKEN')

# google colabを利用している場合は、以下のような形でhugging faceのトークンを読み込める
#from google.colab import userdata
#HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

# 直接notebook上に書き込みたい人は以下のコード
HUGGING_FACE_TOKEN = "hf_************"



# hugging faceへログインする
login(HUGGING_FACE_TOKEN)
os.environ["HF_TOKEN"] = HUGGING_FACE_TOKEN
os.environ["HF_HUB_ENABLE_HF_TRANSFER"]="1"

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/sagemaker-user/.cache/huggingface/token
Login successful


# データの準備

In [5]:
# ダウンロードするURLを指定
URL = "https://raw.githubusercontent.com/yahoojapan/JGLUE/refs/heads/main/datasets/jcommonsenseqa-v1.1/train-v1.1.json"

# 保存するファイル名を指定
save_as = 'jcommonsenseqa.pkl'


response = requests.get(URL)

datas = []

if response.status_code == 200:
    response.encoding = response.apparent_encoding

    # 複数のJSONオブジェクトが連続する場合に対応
    json_text = response.text
    try:
        json_objects = json_text.split('\n')  # または適切なセパレータ
        for obj in json_objects:
            if not obj.strip():  # 空行を無視
                continue
            try:
                data = json.loads(obj)
                datas.append(data)
            except json.JSONDecodeError as e:
                print("それぞれのJSONオブジェクトのデコードエラー:", e)
    except json.JSONDecodeError as e:
        print("全体のJSONデコードエラー:", e)

    with open(save_as, "wb") as f:
        pickle.dump(datas, f)
else:
    print(f"リクエストが失敗しました。ステータスコード: {response.status_code}")

In [6]:
with open(save_as, "rb") as f:
    datas = pickle.load(f)

In [7]:

# データの形式を確認する
sample_dict = datas[0]
print(sample_dict)

{'q_id': 0, 'question': '主に子ども向けのもので、イラストのついた物語が書かれているものはどれ？', 'choice0': '世界', 'choice1': '写真集', 'choice2': '絵本', 'choice3': '論文', 'choice4': '図鑑', 'label': 2}


# JcommonsenseQAをLLMに突っ込むために、`messages`オブジェクトの形式にする

In [8]:
def inputs_and_outputs(datas:list) -> list:
    """
    LLMへインプットするプロンプトとそれに対する解答を作成する。

    - output
        [
            {
                "messages": list,
                "answer": str
            },
        ]
    """
    DEFAULT_SYSTEM_PROMPT = "あなたは日本語で回答するアシスタントです。"
    INSTRUCTION = "質問と回答の選択肢を入力として受け取り、選択肢から回答を選択してください。なお、回答は選択肢の番号（例：0）でするものとします。 回答となる数値をint型で返し、他には何も含めないことを厳守してください。"
    results = []

    for i, sample_dict in enumerate(datas):
        input = f"""
            質問：{sample_dict.get('question', None)}\n
            選択肢：
            0.{sample_dict.get('choice0', None)},
            1.{sample_dict.get('choice1', None)},
            2.{sample_dict.get('choice2', None)},
            3.{sample_dict.get('choice3', None)},
            4.{sample_dict.get('choice4', None)}
        """
        input = input.replace(" ", "") # 不要なスペースを消す。
        answer = sample_dict.get("label", None)

        text = f"以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n{INSTRUCTION}\n\n### 入力:\n{input}\n\n### 応答:\n"

        messages = [
            {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
            {"role": "user", "content": text},
        ]

        result = {"messages": messages, "answer": answer}
        results.append(result)

    return results

list_qa = inputs_and_outputs(datas)

# モデルデータの準備

In [9]:
# フォルダが存在しない場合にのみ作成
MODEL_PATH = "./model/"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

In [10]:
def set_model_path(model_name: str) -> str:
    save_path = MODEL_PATH + model_name.split("/")[-1]
    return save_path


def download_model(model_name: str):
    """LLMをダウンロードする。"""
    save_path = set_model_path(model_name)
    if not os.path.isfile(save_path):
        snapshot_download(
            model_name,
            local_dir=save_path,
        )

        move_cache() # ダウンロードしたモデルのキャッシュを適切な場所に移動します。これにより、モデルの読み込みが効率的に行えるようになる

# ダウンロード済みのモデルとトークナイザの設定と初期化をする

In [11]:
def setup_model(model_name: str) -> dict:
    """
    ダウンロード済みのモデルとトークナイザの設定と初期化をする

    - output
        {
            "tokenizer": AutoTokenizer,
            "llm": LLM, <-- モデルのインスタンス
            "sampling_params": SamplingParams, <-- サンプリングパラメータのインスタンス（テキスト生成で利用する）
        }
    """

    # download the model, if the file is already exist, this function does nothing
    download_model(model_name)

    save_path = set_model_path(model_name)

    # Acceleration with quantization configurations
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Enable 4-bit quantization
        llm_int8_threshold=6.0,  # Int8 mode threshold
        llm_int8_has_fp16_weight=True  # If true, reduce precision of weight to FP16
    )

    # Set up the model
    llm = AutoModelForCausalLM.from_pretrained(
        save_path,
        quantization_config=quantization_config,
        device_map="auto", # Automap to available devices (CPU or GPU)
        low_cpu_mem_usage=True  # Reduce CPU memory usage,
    )
    tokenizer = AutoTokenizer.from_pretrained(save_path)

    # Ensure the model is using CUDA if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    llm.to(device)

    outputs = {
        "tokenizer": tokenizer,
        "llm": llm,
    }

    # モデルをローカルに保存
    llm.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    return outputs


model = setup_model(model_name = model_name)

Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/25.8k [00:00<?, ?B/s]

GatedRepoError: 403 Client Error. (Request ID: Root=1-6768b790-31e9d751118d37f910c72556;f6e823b4-d23d-4a0e-883f-5a1c74707879)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-9b-it/resolve/11c9b309abf73637e4b6f9a3fa1e92e615547819/.gitattributes.
Your request to access model google/gemma-2-9b-it is awaiting a review from the repo authors.

# LLM出力の整形

In [ ]:
def get_answer(result: str, anchor_str: str = "<|im_start|>assistant"):
    """
    出力の中に余計なトークンが含まれるので、それをパースするための処理

    - output: str
    """
    print(f"{result=}")
    index = result.find(anchor_str)
    # インデックスが見つかった場合、インデックスより後の部分を取得
    if index != -1:
        start_position = index + len(anchor_str)
        result = result[start_position:].strip()  # strip()を使って前後の空白を削除
    else:
        result = result


    try:
        _ = int(result) # 整数型で返せているかをチェック
    except Exception as e:
        matches = re.search(r'(?<=assistant).+', result, re.DOTALL)
        result = matches.group(0).strip() if matches else ""

        result = re.findall(r'\d+', result)
        result = result[0]

    return result

# JcommonsenseQAの実行

In [ ]:
def run_llm(
        model: dict,
        list_qa: list,
) -> list:
    """
    DLしたモデルで推論を実行させる。
    """

    
    dict_result = {"model_name": model["llm"].config._name_or_path.split("/")[-1]}
    list_result = []

    def save_temp_result(temp_result: list):
        """
            JcommonsenseQAは8,900以上の問題があり、何らかの理由で途中終了してしまうと、全てやり直しになってしまう.
            （このnotebook上に記載はしていないが）正常に回し切れていない分だけ後で差分実行できるように正常終了した分だけ
            保存しておきたい、というモチベーションで作った関数.
        """
        with open("./tempreults_" + model["llm"].config._name_or_path.split("/")[-1] + '.pkl', "wb") as f:
            pickle.dump(temp_result, f)

    for dict_qa in list_qa:
        messages = dict_qa["messages"]
        answer = dict_qa["answer"]

        input_ids = model["tokenizer"].apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model["llm"].device)

        output_ids = model["llm"].generate(
            input_ids,
            max_new_tokens=1024,
            temperature=0.5,
        )

        llm_answer = model["tokenizer"].decode(output_ids[0], skip_special_tokens=True)
        llm_answer = get_answer(llm_answer)
        llm_answer = int(llm_answer)

        dict_qa["llm_answer"] = llm_answer
        dict_qa["is_correct"] = llm_answer==answer

        list_result.append(dict_qa)
        save_temp_result(list_result)

    try:
        # プログラムの終了前におまじないとして.
        import torch.distributed as dist
        dist.destroy_process_group()
    except Exception as e:
        print(str(e))

    dict_result["qa_results"] = list_result

    return dict_result

run_llm_results = run_llm(model,list_qa[:1])

# 実行結果の保存やロード

In [ ]:
def save_llm_results(run_llm_results: dict) -> None:
    with open("./reults_" + run_llm_results.get("model_name", "not defined") + '.json', 'w', encoding='utf-8') as json_file:
        json.dump(run_llm_results, json_file, ensure_ascii=False, indent=4)


def load_llm_results(model_name: str = model_name) -> dict:
    with open("./reults_" + model_name.split("/")[-1] + '.json', 'r', encoding='utf-8') as json_file:
        return json.load(json_file)

In [ ]:
# LLMの実行結果を保存する
save_llm_results(run_llm_results)

# LLMの実行結果を読み込む
run_llm_results = load_llm_results()

# 実行結果の確認

In [ ]:
run_llm_results.get("model_name", "not defined")
run_llm_results.get("qa_results", "failed to execute")

# 点数のカウント

In [ ]:
qa_results = run_llm_results.get("qa_results", "failed to execute")

counts = 0
for i, res in enumerate(qa_results):
    if res.get("is_correct"):
        counts += 1

score = float(counts)/len(qa_results)
score = '{:.3f}'.format(score)
score = float(score)
run_llm_results["score"] = score

In [ ]:
save_llm_results(run_llm_results)

In [ ]:
# キャッシュクリア
import torch
torch.cuda.empty_cache()